In [1]:
from utils import *
from prompt.response_prompt import *
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from transformers import BertModel, BertTokenizer
import re
from sklearn.preprocessing import StandardScaler


/data/chenxi/anaconda3/envs/myenv/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
/data/chenxi/anaconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /data/chenxi/anaconda3/envs/myenv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/data/chenxi/anaconda3/envs/myenv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: /data/chenxi/anaconda3/envs/myenv did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/data/chenxi/anaconda3/envs/myenv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/data/chenxi/anaconda3/envs/myenv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)
/data/chenxi/anaconda3/envs/myenv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcuda

In [2]:
df = arff_to_dataframe("/data/chenxi/llm-feature-engeneering/dataset/diabetes.arff")

In [3]:

models= {
    # 'Logistic Regression': LogisticRegression(max_iter=1000000),
    # 'K-Nearest Neighbors': KNeighborsClassifier(algorithm='ball_tree'),
    # 'Naive Bayes': GaussianNB(),
    # 'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    # 'AdaBoost': AdaBoostClassifier(),
    # 'Gradient Boosting': GradientBoostingClassifier(),
    # 'Support Vector Machine': SVC(probability=True),  # Enable probability estimates
    # 'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
}

In [4]:
df['analysis'] = pd.read_csv('analysis.csv')
df['sum']= pd.read_csv('sum.csv')

In [5]:
# prompts = df.apply(lambda row: template.format(**row), axis=1)
# df['analysis']= prompts.apply(lambda x: decoder_for_gpt3(x, max_length = 1000))
# # Save the dataframe with the generated summaries to a new CSV file
# df['analysis'].to_csv('analysis.csv', index=False)
# prompts_sum = df['analysis'].apply(lambda row: template_for_sum.format(analysis=row))
# df['sum'] = prompts_sum.apply(lambda x: decoder_for_gpt3(x, max_length = 1000))
# df['sum'].to_csv('sum.csv', index=False)

In [ ]:
# Utility functions
def get_matching_cols(df, regex):
    r = re.compile(regex)
    return list(filter(r.match, df.columns))

def get_embedding_cols(df):
    return get_matching_cols(df, "(vec_\d+)")

def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return openai.Embedding.create(input=[text], model=model)['data'][0]['embedding']

def explode(col, prefix):
    n_cols = len(col[0])
    col_names = [prefix + str(i) for i in range(n_cols)]
    return pd.DataFrame(col.to_list(), columns=col_names)

# Methods for feature engineering
def method_baseline(df):
    X = df[["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal"]]
    y = df["num"]
    scaler = StandardScaler()
    X_final = scaler.fit_transform(X)
    return X_final, y

def method_SelectK(df):
    y = df["num"]
    X_cat = df[["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal"]]
    embed_cols = get_embedding_cols(df)
    X_text = df[embed_cols]
    X_comb = pd.concat([X_cat, X_text], axis=1)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_comb)
    
    # Define a range of possible values of k
    possible_k_values = list(range(1, 50))
    
    best_score = -np.inf
    best_k = None
    best_features = None

    model = SVC(probability=True)

    
    for k in possible_k_values:
        # Use SelectKBest to select top k features
        selector = SelectKBest(mutual_info_classif, k=k)
        X_selected = selector.fit_transform(X_scaled, y)
        
        # Train and evaluate the model using cross-validation
        score = cross_val_score(model, X_selected, y, cv=5, scoring='roc_auc').mean()
        
        # If the score is the best so far, update best_k, best_score, and best_features
        if score > best_score:
            best_score = score
            best_k = k
            best_features = X_selected

    # Combine the categorical data with the best selected features
    X_final = pd.concat([X_cat, pd.DataFrame(best_features)], axis=1)
    X_final.columns = X_final.columns.astype(str)
    print(best_k)
    
    return X_final, y # Also return the best k for information



def method_PCA(df):
    
    # Splitting X and y
    X = df.drop('num', axis=1)
    y = df['num']

    # Separate original categorical features
    X_cat = df[["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal"]]
    # Extract the text embeddings
    embed_cols = get_embedding_cols(X)
    X_text = X[embed_cols]

    # Combine the embeddings and the original set
    X_comb = pd.concat([X_cat, X_text], axis=1)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_comb)

    # Applying PCA on the combined data
    best_n_components = None
    best_score = float('-inf')
    for n_components in range(1, 50):  # Checking all possible number of components
        pca = PCA(n_components=n_components)
        X_pca = pca.fit_transform(X_scaled)

        # Train a model (e.g., logistic regression) on the PCA components and compute the performance
        model = LogisticRegression()
        score = cross_val_score(model, X_pca, y, cv=5, scoring='roc_auc').mean()

        if score > best_score:
            best_score = score
            best_n_components = n_components

    # Use PCA with the best number of components
    pca = PCA(n_components=best_n_components)
    X_pca = pca.fit_transform(X_scaled)

    # Combining PCA components with original categorical data
    X_final = pd.concat([X_cat, pd.DataFrame(X_pca)], axis=1)
    X_final.columns = X_final.columns.astype(str)

    return X_final, y


# Main evaluation function
def evaluate_models(df, models, methods):
    # logic to evaluate models with provided methods
    colors = ['black', 'green', 'blue', 'red']

    for metric in ['accuracy', 'roc_auc']:
        plt.figure(figsize=(15, 10))

        # Iterate through methods
        for i, method in enumerate(methods):
            if method == 'baseline':
                X_final, y = method_baseline(df)
            elif method == 'PCA':
                X_final, y = method_PCA(df)
            elif method == 'SelectK':
                X_final, y = method_SelectK(df)
        
            # ... (rest of the evaluation function logic)
            kfold = KFold(n_splits=5, shuffle=True, random_state=42)
            performance_metrics = {metric: {model_name: cross_val_score(model, X_final, y, cv=kfold, scoring=metric) for model_name, model in models.items()}}

            for name, scores in performance_metrics[metric].items():
                print(f'{name}: {metric}: {scores.mean()} ± {scores.std()}')

            x_ticks_positions = np.arange(len(models)) + i * 0.2
            plt.boxplot([performance_metrics[metric][model_name] for model_name in models.keys()], positions=x_ticks_positions, widths=0.2, patch_artist=True,
                        boxprops=dict(facecolor=colors[i], color=colors[i], alpha=0.6),
                        capprops=dict(color=colors[i]),
                        whiskerprops=dict(color=colors[i]),
                        flierprops=dict(color=colors[i], markeredgecolor=colors[i]),
                        medianprops=dict(color='black'))

        plt.legend(handles=[mpatches.Patch(color=colors[i], label=methods[i]) for i in range(len(methods))], loc='upper right')
        plt.title(f"Model performance ({metric})")
        plt.ylabel(metric)
        plt.xticks(ticks=np.arange(len(models)), labels=models.keys())
        plt.show()


methods = ['baseline', 'SelectK', 'PCA']
evaluate_models(df, models, methods)

<bound method NDFrame.head of      Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  Outcome  
0                       0.627   50 